In [1]:
import pandas as pd
import numpy as np
import scipy.stats as sps

In [2]:
data = pd.read_csv("./titanic.csv", sep=",", index_col=0)
data

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S


In [4]:
df2 = data[["Survived", "Pclass"]]
selection2_c1 = df2.where(
    df2.Pclass == 1).dropna().Survived.values.astype("int64")
selection2_c2 = df2.where(
    df2.Pclass == 2).dropna().Survived.values.astype("int64")
selection2_c3 = df2.where(
    df2.Pclass == 3).dropna().Survived.values.astype("int64")

print("a)")
# h0 - шанс выжить у Pclass1 == Pclass3
# ha - шанс выжить у Pclass1 > Pclass3

Pclass3Survived = data['Pclass'].where(
    data['Survived'] == 1).dropna().where(data['Pclass'] == 3).dropna()
Pclass3Propability = len(Pclass3Survived) / \
    len(data['Pclass'].where(data['Pclass'] == 3).dropna())
T = np.sqrt(len(selection2_c1)) * (selection2_c1.mean() -
                                   Pclass3Propability)/selection2_c1.std()
n = len(selection2_c1)
if T < sps.t.ppf(0.95, n - 1):
    print("Гипотеза H0 верна => шанс выжить у Pclass1 == Pclass3")
else:
    print("Гипотеза H0 отвергнута в пользу Ha => шанс выжить у Pclass1 > Pclass3")

a)
Гипотеза H0 отвергнута в пользу Ha => шанс выжить у Pclass1 > Pclass3


In [5]:
print("b)")
# h0 - E1 = E2 = E3
# ha - E1, E2, E3 различны
count_surv = data.Survived.sum()

E = [len(selection2_c1)/3, len(selection2_c2)/3, len(selection2_c3)/3]

O = [selection2_c1.mean(), selection2_c2.mean(), selection2_c3.mean()]

Q = 0
for i in range(len(O)):
    Q += ((O[i] - E[i]) ** 2)/E[i]

if Q < sps.chi2.ppf(0.95, 2):
    print("Гипотеза H0 верна => E1 = E2 = E3")
else:
    print("Гипотеза H0 отвергнута в пользу Ha => E1, E2, E3 различны")

b)
Гипотеза H0 отвергнута в пользу Ha => E1, E2, E3 различны


In [10]:
print("c)")
# h0 - цена билета имеет нормальное распределение
# ha - цена билета имеет не нормальное распределение

fare = data.Fare

mean_ml = fare.mean()
var_ml = fare.std()

O = []
E = []
for x in fare.unique():
    O.append(data.Fare.where(data.Fare == x).count()/fare.count())
    E.append(sps.norm.pdf(x, mean_ml, var_ml))

Q = 0
for i in range(len(O)):
    Q += ((O[i] - E[i]) ** 2)/E[i]

if Q < sps.chi2.ppf(0.95, len(O)-1):
    print("Гипотеза H0 верна => цена билета имеет нормальное распределение N(mean_ml, var_ml)")
else:
    print("Гипотеза H0 отвергнута в пользу Ha => цена билета имеет не нормальное распределение")

c)
Гипотеза H0 отвергнута в пользу Ha => цена билета имеет не нормальное распределение


In [7]:
print("d)")
# h0 - корреляиця равна 0
# ha - корреляция больше 0

age = data.Age.fillna(0)
fare = data.Fare

norm_age = (age - age.mean())/age.std()
norm_fare = (fare - fare.mean())/fare.std()

n = len(norm_age)
tmp1 = 0
tmp2 = 0
tmp3 = 0

for i in range(1, n+1):
    tmp1 += norm_age[i]*norm_fare[i]
    tmp2 += norm_fare[i]**2
    tmp3 += norm_age[i]**2
p_n = (tmp1/n) * (1/np.sqrt(tmp2/n)) * (1/np.sqrt(tmp3/n))
H = 1/2*(np.log(1 + p_n) - np.log(1 - p_n))
q = sps.norm.ppf(0.975, 0, 1)
b = 1/np.sqrt(n)
left = (np.exp(2*(H - q*b)) - 1)/(np.exp(2*(H - q*b)) + 1)
right = (np.exp(2*(H + q*b)) - 1)/(np.exp(2*(H + q*b)) + 1)
print(f"Доверительный инетрвал для корреляции: [{left}, {right}]")
if 0 >= left and 0 <= right:
    print("Гипотеза H0 верна => корреляция равна 0")
else:
    print("Гипотеза H0 отвергнута в пользу Ha => корреляция существует")

d)
Доверительный инетрвал для корреляции: [0.07057588436623875, 0.19931195424652418]
Гипотеза H0 отвергнута в пользу Ha => корреляция существует
